In [20]:
!python3 --version
import tensorflow as tf
print("Tensorflow", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Creates a session with device placement logs
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
print(sess)

DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage
import skimage.draw
import cv2
import random
import math
import re
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
import tifffile as tiff

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils
#from custom_multi import nms_suppression_multi
from custom import CustomConfig, CustomDataset
from postprocessing import nms_suppression_multi, crop_from_results

Python 3.9.19
Tensorflow 2.15.0
Num GPUs Available:  0
Device mapping: no known devices.


In [21]:
ROOT_DIR = os.getcwd()
sys.path.append(ROOT_DIR)  # To find local version of the library
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

## Params

In [37]:
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0
num_gpu = 1 # Number of GPUs to use
num_img_per_gpu = 1 # Batch size
detection_min_confidence = 0.7 # Minimum confidence level for detection
detection_nms_threshold = 0.3 # NMS threshold for detection

#weights_subpath = 'multicell20240417T1006/mask_rcnn_multicell_0050.h5'
#weights_subpath = 'grayscaleMulti/mask_rcnn_multicell_0050.h5'
weights_subpath = 'RGBmultiBest/mask_rcnn_multicell_0050.h5'
results_name = 'BZ_Test_gray'

#TEST_DIR = '/Users/tom/Desktop/Stanford/RA/OligodendroSight/OL_mrcnn/data/test/imgsnorm'
TEST_DIR = '/Users/tom/Downloads/BZ_OL_images_tiff_norm'

VISUALIZE = True

In [38]:
class InferenceConfigMulti(CustomConfig):
    GPU_COUNT = num_gpu
    IMAGES_PER_GPU = num_img_per_gpu
    DETECTION_MIN_CONFIDENCE = detection_min_confidence #Minimum probability value to accept a detected instance
    DETECTION_NMS_THRESHOLD = detection_nms_threshold # Non-maximum suppression threshold for detection

inference_config_multi = InferenceConfigMulti()
macro_model = modellib.MaskRCNN(mode="inference",
                            config=inference_config_multi,
                            model_dir=DEFAULT_LOGS_DIR)

MACRO_WEIGHTS_SUBPATH = weights_subpath
macro_model_path = os.path.join(DEFAULT_LOGS_DIR, MACRO_WEIGHTS_SUBPATH)

print("Loading macro weights from ", macro_model_path)
#macro_model.load_weights(macro_model_path, by_name=True) #deprecated on MacOS
tf.keras.Model.load_weights(macro_model.keras_model, macro_model_path , by_name=True, skip_mismatch=True)

RESULTS_NAME = results_name
RESULTS_DIR = os.path.join(ROOT_DIR, "results", RESULTS_NAME)
os.makedirs(RESULTS_DIR, exist_ok=True)

Loading macro weights from  /Users/tom/Desktop/Stanford/RA/OligodendroSight/OL_mrcnn/logs/RGBmultiBest/mask_rcnn_multicell_0050.h5


2024-06-02 20:44:20.939298: W tensorflow/c/c_api.cc:305] Operation '{name:'mrcnn_mask_bn1_7/beta/Assign' id:75559 op device:{requested: '', assigned: ''} def:{{{node mrcnn_mask_bn1_7/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](mrcnn_mask_bn1_7/beta, mrcnn_mask_bn1_7/beta/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [31]:
# Validation dataset
dataset_val = CustomDataset()
dataset_val.load_custom(os.path.join(ROOT_DIR,"data"), "valid")
dataset_val.prepare()

In [40]:
image_paths = []
for filename in os.listdir(TEST_DIR):
    if filename.endswith(".tif"):
        image_paths.append(os.path.join(TEST_DIR, filename))

res_list = []
for image_path in image_paths:
    print(os.path.basename(image_path))
    image = skimage.io.imread(image_path)
    gray_image = skimage.io.imread(image_path, as_gray=True)
    gray_image = (gray_image * 255).astype(np.uint8)
    image = np.stack((gray_image,)*3, axis=-1)
    macro_results = macro_model.detect([image], verbose=0)
    macro_results = nms_suppression_multi(macro_results, 0.3)
    r = macro_results[0]
    if VISUALIZE:
        display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                        dataset_val.class_names, r['scores'], 
                        title="Predictions",
                        show_bbox=True, show_mask=False)
    img_res_dir = os.path.join(RESULTS_DIR, os.path.basename(image_path)[:-4])
    os.makedirs(img_res_dir, exist_ok=True)
    res_list = crop_from_results(image_path, image, img_res_dir, macro_results, res_list, dataset_val.class_names, modify_results=True)

res_df = pd.DataFrame(res_list, columns=['image_name', 'detection_id', 'class', 'score', 'bbox'])
res_df.to_csv(os.path.join(RESULTS_DIR, 'results.csv'), index=False)

BZ-MBL_002_20x_LatA2uM-right-28.tif


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (3,2)  and requested shape (2,2)